In [10]:
import mysql.connector

yespa_db = mysql.connector.connect(
    host = 'database-1.c3ieyo8m4v9o.ap-northeast-2.rds.amazonaws.com',
    port = 3306,
    user = 'eda_team',
    database = 'eda_project',
    password = '***'
)

cur = yespa_db.cursor(buffered=True, dictionary=True)

cur.execute('SELECT * FROM GIHU_metro_coor')
result1 = cur.fetchall()

cur.execute('SELECT * FROM NOTGIHU_metro_coor')
result2 = cur.fetchall()
    
yespa_db.close()

/home/jinsa/.local/lib/python3.10/site-packages/mysql/connector/abstracts.py:130: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if group is 'connector_python':
/home/jinsa/.local/lib/python3.10/site-packages/mysql/connector/abstracts.py:130: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if group is 'connector_python':


ProgrammingError: 1045 (28000): Access denied for user 'eda_team'@'211.197.28.14' (using password: YES)

In [ ]:
coor_cityhall = [37.5665851, 126.9782038]

In [ ]:
import pandas as pd 

df1 = pd.DataFrame(result1)
df2 = pd.DataFrame(result2)

In [ ]:
from haversine import haversine

distance = haversine((df1['위도'][0],df1['경도'][0]),coor_cityhall,unit='km')
distance

11.408497678776548

In [ ]:
from haversine import haversine


df1['시청과의거리'] = df1.apply(lambda row: haversine((row['위도'], row['경도']), coor_cityhall, unit='km'), axis=1)
df2['시청과의거리'] = df2.apply(lambda row: haversine((row['위도'], row['경도']), coor_cityhall, unit='km'), axis=1)

In [ ]:
df =pd.concat([df1,df2])
df.reset_index(drop=True, inplace=True)
df

,역명,호선,위도,경도,시청과의거리
0,창동,1호선,37.653181,127.047663,11.408498
1,월계,1호선,37.633114,127.058835,10.256009
2,외대앞,1호선,37.596157,127.063572,8.209979
3,대방,1호선,37.513392,126.926407,7.472727
4,신길,1호선,37.517018,126.917743,7.667731
...,...,...,...,...,...
760,구리,중앙선,37.603400,127.144000,15.172069
761,개포동,분당선,37.489100,127.066000,11.583473
762,별내별가람,진접선,37.667800,127.116000,16.552017
763,오남,진접선,37.705000,127.193000,24.385025


In [ ]:
df.to_csv('distance_data.csv', index=False)

In [ ]:
from geopy.geocoders import Nominatim

address=[]

def get_address_from_coordinates(latitude, longitude):
    geolocator = Nominatim(user_agent="geoapiExercises")
    location = geolocator.reverse((latitude, longitude), language='ko')
    return location.address if location else "주소를 찾을 수 없습니다."

for idx, row in df.iterrows():
    latitude = row['위도']
    longitude = row['경도']
    
    address.append(get_address_from_coordinates(latitude, longitude))
    
address



GeocoderInsufficientPrivileges: Non-successful status code 403

In [ ]:
df['주소']= address
df

,역명,호선,위도,경도,시청과의거리,주소
0,NaN,NaN,NaN,NaN,NaN,"77, 마들로11길, 창4동, 도봉구, 서울, 01418, 대한민국"
1,NaN,NaN,NaN,NaN,NaN,"1, 월계로53길, 월계동, 월계2동, 노원구, 서울, 01885, 대한민국"
2,NaN,NaN,NaN,NaN,NaN,"1, 휘경로5길, 이문동, 이문2동, 동대문구, 서울, 02403, 대한민국"
3,NaN,NaN,NaN,NaN,NaN,"3, 여의대방로, 신길7동, 영등포구, 서울, 07319, 대한민국"
4,NaN,NaN,NaN,NaN,NaN,"3, 노들로, 신길1동, 영등포구, 서울, 07319, 대한민국"
...,...,...,...,...,...,...
760,NaN,NaN,NaN,NaN,NaN,"2, 34, 구리시, 경기도, 11922, 대한민국"
761,NaN,NaN,NaN,NaN,NaN,"개포로, 개포동, 개포2동, 강남구, 서울, 06329, 대한민국"
762,NaN,NaN,NaN,NaN,NaN,"덕송2로, 별내동, 남양주시, 경기도, 12091, 대한민국"
763,NaN,NaN,NaN,NaN,NaN,"921, 진건오남로, 오남읍, 남양주시, 경기도, 12036, 대한민국"


In [ ]:
df.to_csv('address_data.csv', index=False)

In [ ]:
df=pd.read_csv('/home/jinsa/dev_ws/EDA_PROJECT/visualization/Data_union/address_data.csv')

In [ ]:
filtered_df = df[df['주소'].str.contains('고양시|용인시|부천시')]
filtered_df.reset_index(drop=True, inplace=True)
filtered_df

,역명,호선,위도,경도,시청과의거리,주소
0,지축,3호선,37.648281,126.912551,10.768912,"안진1길, 효자동, 덕양구, 고양시, 경기도, 10585, 대한민국"
1,소새울,서해선,37.468500,126.797000,19.348424,"소새울, 소사로, 소사본동, 소사구, 부천시, 경기도, 14901, 대한민국"
2,소사,서해선,37.483300,126.795000,18.622287,"소사로, 소사동, 원미구, 부천시, 경기도, 14651, 대한민국"
3,전대.에버랜드,에버라인선,37.285300,127.220000,37.870493,"전대·에버랜드, 에버랜드로, 처인구, 용인시, 경기도, 17028, 대한민국"
4,둔전,에버라인선,37.267100,127.214000,39.276280,"둔전, 경안천로, 처인구, 용인시, 경기도, 17038, 대한민국"
...,...,...,...,...,...,...
61,곡산,경의중앙선,37.645700,126.802000,17.841643,"곡산역, 160, 경의로, 백석1동, 일산동구, 고양시, 경기도, 10418, 대한민국"
62,능곡,경의중앙선,37.618800,126.821000,15.018489,"1, 토당로42번길, 행주동, 덕양구, 고양시, 경기도, 10510, 대한민국"
63,행신,경의중앙선,37.612100,126.834000,13.676847,"3, 소원로, 행신2동, 덕양구, 고양시, 경기도, 10523, 대한민국"
64,강매,경의중앙선,37.612300,126.843000,12.952161,"3, 강매교, 강매동, 행신2동, 덕양구, 고양시, 경기도, 10531, 대한민국"


In [ ]:
dff1 = filtered_df[filtered_df['주소'].str.contains('고양시')]
dff2 = filtered_df[filtered_df['주소'].str.contains('용인시')]
dff3 = filtered_df[filtered_df['주소'].str.contains('부천시')]

In [ ]:
dff1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 0 to 65
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   역명      20 non-null     object 
 1   호선      20 non-null     object 
 2   위도      20 non-null     float64
 3   경도      20 non-null     float64
 4   시청과의거리  20 non-null     float64
 5   주소      20 non-null     object 
dtypes: float64(3), object(3)
memory usage: 1.1+ KB


In [ ]:
average_distance_to_city_hall = [dff1['시청과의거리'].mean(),dff2['시청과의거리'].mean(),dff3['시청과의거리'].mean()]
average_distance_to_city_hall



[17.208202598893784, 35.65958299074703, 18.72635326313178]

In [ ]:
# 거리 간의 오차 +30%

metro_average_distance= [dff1['시청과의거리'].mean()*13/10,dff2['시청과의거리'].mean()*13/10,dff3['시청과의거리'].mean()*13/10]
metro_average_distance

[22.37066337856192, 46.357457887971144, 24.344259242071313]

In [ ]:
gyeonggi = [
    '가평시', '고양시', '과천시', '광명시', '광주시', '구리시', '군포시', '김포시',
    '남양주시', '동두천시', '부천시', '성남시', '수원시', '시흥시', '안산시', '안성시',
    '안양시', '양주시', '양평시', '여주시', '연천시', '오산시', '용인시', '의왕시',
    '의정부시', '이천시', '파주시', '평택시', '포천시', '하남시', '화성시'
]


incheon = [
    '강화군', '계양구', '남동구', '동구', '미추홀구', '부평구', '서구', '연수구',
    '옹진군', '중구'
]

seoul = [
    '강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구',
    '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구',
    '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구',
    '중랑구']

total = gyeonggi+incheon

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 765 entries, 0 to 764
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   역명      765 non-null    object 
 1   호선      765 non-null    object 
 2   위도      765 non-null    float64
 3   경도      765 non-null    float64
 4   시청과의거리  765 non-null    float64
 5   주소      765 non-null    object 
dtypes: float64(3), object(3)
memory usage: 36.0+ KB


In [ ]:
# 각 시별로 평균 시청과의 거리 계산 + 오차 고려
average_distance_to_city_hall = [df[df['주소'].str.contains(area)]['시청과의거리'].mean()*13/10 for area in gyeonggi + incheon]

average_distance_to_city_hall

result_df = pd.DataFrame({
    '시군구': gyeonggi + incheon,
    '평균거리(+30%)': average_distance_to_city_hall
})

result_df

,시군구,평균거리(+30%)
0,가평시,NaN
1,고양시,22.370663
2,과천시,18.764303
3,광명시,20.464367
4,광주시,43.168425
5,구리시,19.090902
6,군포시,31.040381
7,김포시,34.893751
8,남양주시,29.069331
9,동두천시,50.284193


In [ ]:
result_df.to_csv('sigungu_average_distance.csv', index=False)